In [1]:
# disply .tif w/ rasterio example:
# https://github.com/USF-IMARS/l2-processing/blob/main/Example_LP_DAAC_notebooks/Topic_1__AWS_Data_Access.ipynb

In [2]:
# netcdf viewing

from netCDF4 import Dataset
import numpy as np

my_example_nc_file = './MODA_OC_py_data/A2007143182500.L2_LAC_OC.x.nc'
with Dataset(my_example_nc_file, mode='r') as filehandle:
    # display(filehandle.variables)
    # display(filehandle)
    # lons = filehandle.variables['lon'][:]
    # tmax_units = filehandle.variables['Tmax'].units
    
    import matplotlib.pyplot as plt
    from mpl_toolkits.basemap import Basemap
    # Get some parameters for the Stereographic Projection
    lon_0 = filehandle.start_center_longitude
    lat_0 = filehandle.start_center_latitude

    m = Basemap(
        width=5000000,height=3500000,
        resolution='l',projection='stere',
        lat_ts=40,
        lat_0=lat_0,
        lon_0=lon_0
    )

KeyError: 'PROJ_LIB'

In [ ]:
conda install proj4

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 